In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import nltk 
#nltk.download()
import string 
import re 
from nltk.tokenize import PunktSentenceTokenizer
from collections import Counter 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np 
#import matplotlib.pyplot as plt 
#import seaborn as sns 
#% matplotlib inline
from operator import itemgetter
from nltk.util import ngrams

csv = 'C:\Users\erik.larson\Downloads\merged_data.csv'

merged_data_csv = pd.read_csv(csv, encoding='utf-8', usecols=['Story_Text', 'email_Submmitted', 'votes_total'])
stop_words = set(stopwords.words('english'))
merged_data_csv.dropna(inplace=True)
story_text = merged_data_csv.Story_Text.tolist()
words = []

ps = PorterStemmer()

for s in story_text:
    words.append(word_tokenize(s))
    
filtered_sentences = []
        
for text in story_text:
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    text = text.replace('H.E.A.R.T', 'Health, Education, and Research in Tibet')
    text = text.replace('P.R.O.U.D', 'Pet Rescue of Unwanted Dogs')
    text = text.replace('C.H.A.N.G.E', 'ChangeInc')
    text = re.sub('['+string.punctuation+']', ' ', text)
    text = text.replace('\\n', '')
    filtered_sentences.append(text)
    
stemmed = []

for text in filtered_sentences:
    text = ''.join([ps.stem(word) for word in text])
    stemmed.append(text)
    
stemmed_sentences = []

for words in filtered_sentences:
    text = ' '.join([ps.stem(w) for w in words.split()])
    stemmed_sentences.append(text)

train = stemmed_sentences[:576]
train = ''.join(train)
custom_sent_tokenizer = PunktSentenceTokenizer(train)

chunk_list = []

def process_content():
    try:
        for i in tokenized:
            words = nltk.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            tagged = list(tagged)
            chunk_list.append(tagged)
            return chunk_list

    except Exception as e:
        print(str(e))
        
for sentence in stemmed_sentences[576:]:
    tokenized = custom_sent_tokenizer.tokenize(sentence)
    process_content()
    
chunks = process_content()
count_list = []
for chunk in chunks:
    counts = Counter(tag for word, tag in chunk)
    count_list.append(counts)

merged = merged_data_csv[576:]
counts = pd.DataFrame.from_dict(count_list)
merged.reset_index(inplace=True)
final = pd.merge(merged, counts, left_index=True, right_index=True)

In [39]:
final = final.fillna(0)


flattened = [val for sublist in chunks for val in sublist]
df = pd.DataFrame.from_dict(flattened)
df[0].value_counts()

sorted_list = {}

for column in final.columns:
    if(final[column].dtype) != object:
        sorted_list[column] = final[column].sum()
        
sorted([(value,key) for (key,value) in sorted_list.items()])

vcols = []
ncols = []

for column in final.columns:
    if str(column)[0] == 'V':
        vcols.append(column)
        
for column in final.columns:
    if str(column)[0] == 'N':
        ncols.append(column)        

for column in final.columns:
    if str(column)[0] == 'J':
        ncols.append(column)   

final[vcols]

#final['Verbs'] = final['VB'] + final.VBD + final.VBG + final.VBN + final.VBP + final.VBZ
#final['Nouns'] = final.NN + final.NNP + final.NNPS + final.NNS
#final['Descriptors'] = final.JJ + final.JJR + final.JJS + final.RB + final.RBR + final.RBS
#final = final[['Verbs', 'Nouns', 'Descriptors', 'CD', 'email_Submmitted', 'votes_total']]
final = final.sort_values(by='votes_total', ascending=False)
#final.Verbs.plot(kind='hist')
#final.Nouns.plot(kind='hist')
#final.Descriptors.plot(kind='hist')
final['A:V ratio'] = final.Descriptors / final.Verbs
final.sort_values(by='A:V ratio', ascending=False)

,Verbs,Nouns,Descriptors,CD,email_Submmitted,votes_total,A:V ratio
178,0.0,3.0,4.0,0.0,wetteee9@yahoo.com,3.0,inf
41,3.0,24.0,10.0,0.0,larisa_w@hotmail.com,3.0,3.333333
48,5.0,73.0,16.0,0.0,emilytakoudes@gmail.com,5.0,3.200000
110,7.0,78.0,22.0,1.0,jennybel@mchsi.com,0.0,3.142857
64,9.0,30.0,27.0,3.0,sueschrad@hotmail.com,1.0,3.000000
226,9.0,62.0,26.0,0.0,nzegbugordian@gmail.com,1.0,2.888889
116,6.0,63.0,17.0,2.0,ladedah611@aol.com,1.0,2.833333
112,8.0,44.0,22.0,1.0,ericav9@comcast.net,15.0,2.750000
14,6.0,40.0,16.0,2.0,collinpuri@yahoo.com.sg,0.0,2.666667
7,5.0,44.0,13.0,0.0,julia.diegmann@live.com,0.0,2.600000


In [40]:
# Correlation matrix

final.corr()

,Verbs,Nouns,Descriptors,CD,votes_total,A:V ratio
Verbs,1.000000,0.867015,0.932315,0.693350,0.084016,-0.107751
Nouns,0.867015,1.000000,0.924208,0.693067,0.075419,0.096247
Descriptors,0.932315,0.924208,1.000000,0.680304,0.049434,0.165483
CD,0.693350,0.693067,0.680304,1.000000,0.020442,-0.014270
votes_total,0.084016,0.075419,0.049434,0.020442,1.000000,-0.073612
A:V ratio,-0.107751,0.096247,0.165483,-0.014270,-0.073612,1.000000


In [38]:
sentiment_df = pd.read_csv('C:\Users\erik.larson\Downloads\\finaldatadive.csv', usecols=['email_Submmitted', 'SentimentScore'])

giving_story_df = pd.merge(sentiment_df, final, on='email_Submmitted')
giving_story_df = giving_story_df.sort_values(by='votes_total', ascending=False)
giving_story_df['Sentiment:Verb'] = giving_story_df['SentimentScore'] / giving_story_df['Verbs']
giving_story_df.sort_index()

,email_Submmitted,SentimentScore,Verbs,Nouns,Descriptors,CD,votes_total,A:V ratio,Sentiment:Verb
0,connordavis@sbcglobal.net,1.000000e+00,27.0,70.0,36.0,7.0,0.0,1.333333,3.703704e-02
1,sayma.hai@gmail.com,9.994990e-01,9.0,20.0,8.0,1.0,1.0,0.888889,1.110554e-01
2,fenojleslie@yahoo.in,9.999790e-01,35.0,81.0,26.0,3.0,3.0,0.742857,2.857083e-02
3,jennybel@mchsi.com,9.995770e-01,7.0,78.0,22.0,1.0,0.0,3.142857,1.427967e-01
4,brat296405@yahoo.com,1.000000e+00,5.0,13.0,8.0,0.0,8.0,1.600000,2.000000e-01
5,megankyis@gmail.com,2.269000e-03,10.0,26.0,9.0,0.0,80.0,0.900000,2.269000e-04
6,wolfheart4499@gmail.com,9.392490e-01,10.0,31.0,4.0,2.0,12.0,0.400000,9.392490e-02
7,blapee@yahoo.com,9.961490e-01,36.0,121.0,35.0,3.0,534.0,0.972222,2.767081e-02
8,jenniferhope77@gmail.com,9.998850e-01,30.0,72.0,33.0,1.0,10.0,1.100000,3.332950e-02
9,southerngirl1973@hotmail.com,1.000000e+00,14.0,44.0,16.0,1.0,1.0,1.142857,7.142857e-02


In [35]:
def ngrammer(k, quantile):

    for s in stemmed_sentences:
        n = k
        threegrams = ngrams(s.split(), n)
        grams_list.append(threegrams)

    for gram in grams_list:
        final_list.append(list(gram))

    flattened = [val for sublist in final_list for val in sublist]
    df = pd.DataFrame.from_dict(Counter(flattened).most_common())
    indexer = int(len(df) * quantile)
    return df[ : indexer]

ngrammer(3, .005)

,0,1
0,"(non, profit, organ)",66
1,"(didn, t, know)",54
2,"(www, facebook, com)",50
3,"(Wildlif, Center, Virginia)",44
4,"(give, back, commun)",40
5,"(Neonat, Womb, Foundat)",38
6,"(Way, Oxford, Counti)",36
7,"(Unit, Way, Oxford)",36
8,"(make, differ, live)",34
9,"(Brain, Aneurysm, Foundat)",32


In [36]:
ngrammer(2, .005)

,0,1
0,"(give, back)",190
1,"(year, old)",181
2,"(non, profit)",156
3,"(year, ago)",130
4,"(http, www)",121
5,"(I, m)",112
6,"(didn, t)",110
7,"(make, differ)",110
8,"(inspir, give)",109
9,"(high, school)",99
